# Домашнє завдання: Внесення оновлень в БД і робота з транзакціями

Це ДЗ передбачене під виконання на локальній машині. Виконання з Google Colab буде суттєво ускладнене.

## Підготовка
1. Переконайтесь, що у вас встановлены необхідні бібліотеки:
   ```bash
   pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv
   ```

2. Створіть файл `.env` з параметрами підключення до бази даних classicmodels. Базу даних ви можете отримати через

  - docker-контейнер згідно існтрукції в [документі](https://www.notion.so/hannapylieva/Docker-1eb94835849480c9b2e7f5dc22ee4df9), також відео інструкції присутні на платформі - уроки "MySQL бази, клієнт для роботи з БД, Docker і ChatGPT для запитів" та "Як встановити Docker для роботи з базами даних без терміналу"
  - або встановивши локально цю БД - для цього перегляньте урок "Опціонально. Встановлення MySQL та  БД Сlassicmodels локально".
  
  Приклад `.env` файлу ми створювали в лекції. Ось його обовʼязкове наповнення:
    ```
    DB_HOST=your_host
    DB_PORT=3306 або 3307 - той, який Ви налаштували
    DB_USER=your_username
    DB_PASSWORD=your_password
    DB_NAME=classicmodels
    ```
  Якщо ви створили цей файл під час перегляду лекції - **новий створювати не треба**. Замініть лише назву БД, або пропишіть назву в коді створення підключення (замість отримання назви цільової БД зі змінних оточення). Але переконайтесь, що до `.env` файл лежить в тій самій папці, що і цей ноутбук.

  **УВАГА!** НЕ копіюйте скрит для **створення** `.env` файлу. В лекції він наводиться для прикладу. І давалось пояснення, що в реальних проєктах ми НІКОЛИ не пишемо доступи до бази в коді. Копіювання скрипта для створення `.env` файлу сюди в ДЗ буде вважатись грубою помилкою і ми зніматимемо бали.

3. Налаштуйте підключення через SQLAlchemy до БД за прикладом в лекції.

Рекомендую вивести (відобразити) змінну engine після створення. Вона має бути не None! Якщо None - значить у Вас не підтягнулись налаштування з .env файла.

Ви також можете налаштувати параметри підключення до БД без .env файла, просто прописавши текстом в відповідних місцях. Це - не рекомендований підхід.



In [1]:
!pip install sqlalchemy pymysql openpyxl requests python-dotenv --quiet

In [2]:
from dotenv import load_dotenv
load_dotenv("HW_12_1.env", override=True)  

True

In [3]:
import os, pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL

load_dotenv("HW_12_1.env", override=True)

url = URL.create(
    "mysql+pymysql",
    username=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    database=os.getenv("DB_NAME"),
    query={"charset": "utf8mb4"},
)
engine = create_engine(url, pool_pre_ping=True)

with engine.connect() as cn:
    print("SELECT 1 ->", cn.execute(text("SELECT 1")).scalar())

pd.read_sql(text("SELECT * FROM customers LIMIT 5"), engine)

SELECT 1 -> 1


,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166,71800.0
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.0
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370,118200.0
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504,81700.0


## Завдання

### Завдання 1: Оновлення інформації про клієнта (2 бали)

**Створіть функцію для оновлення контактної інформації клієнта за його номером** з наступними можливостями:
- Оновлення телефону клієнта
- Оновлення email (якщо поле існує в таблиці)

Опціонально, якщо вам хочеться більше практики:
- Логування змін в окрему таблицю

Використайте підхід з параметризованими запитами через `text()` та `UPDATE` оператор. Не забудьте на початку перевірити чи існує клієнт з таким номером в базі - це хороша практика.

Отримати всі колонки, які існують в таблиці ви можете наступним запитом
```
  SELECT COLUMN_NAME, DATA_TYPE
  FROM INFORMATION_SCHEMA.COLUMNS
  WHERE TABLE_NAME = 'customers'
```

Запустіть функцію і продемонструйте її роботу, запустивши SELECT, який допоможе це зробити.


In [4]:
cols = pd.read_sql(
    text("""
      SELECT COLUMN_NAME
      FROM INFORMATION_SCHEMA.COLUMNS
      WHERE TABLE_SCHEMA = :db AND TABLE_NAME = 'customers'
    """),
    engine, params={"db": os.getenv("DB_NAME")}
)
cols

,COLUMN_NAME
0,addressLine1
1,addressLine2
2,city
3,contactFirstName
4,contactLastName
5,country
6,creditLimit
7,customerName
8,customerNumber
9,phone


In [22]:
def update_customer_both(customer_number: int, phone: str, email: str):
    with engine.begin() as cn:
        if not cn.execute(text("SELECT 1 FROM customers WHERE customerNumber=:id"),
                          {"id": customer_number}).scalar():
            raise ValueError(f"Клієнта {customer_number} не знайдено")

        has_email = cn.execute(text("""
            SELECT 1 FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = DATABASE()
              AND TABLE_NAME='customers' AND COLUMN_NAME='email'
        """)).scalar()

        if has_email:
            cn.execute(text("""
                UPDATE customers SET phone=:phone, email=:email
                WHERE customerNumber=:id
            """), {"id": customer_number, "phone": phone, "email": email})
        else:
        
            cn.execute(text("""
                UPDATE customers SET phone=:phone
                WHERE customerNumber=:id
            """), {"id": customer_number, "phone": phone})
            print("Колонки 'email' немає: оновлено лише телефон.")


In [23]:
pd.read_sql(text("""
    SELECT customerNumber, customerName, phone
    FROM customers WHERE customerNumber = :id
"""), engine, params={"id": 103})

update_customer(103, phone="+380634565678", email="abs@gmail.com")

pd.read_sql(text("""
    SELECT customerNumber, customerName, phone
    FROM customers WHERE customerNumber = :id
"""), engine, params={"id": 103})

Оновлено phone для клієнта 103


,customerNumber,customerName,phone
0,103,Atelier graphique,+380634565678


### Завдання 2: Створення нового замовлення з транзакцією (5 балів)

**Реалізуйте процес створення нового замовлення** з наступними кроками в одній транзакції:
- Створення запису в таблиці `orders`
- Додавання товарних позицій в `orderdetails`
- Перевірка наявності товарів на складі
- Зменшення кількості товарів на складі

Запустіть процес з тестовими даними і продемонструйте через SELECT, що процес успішно відпрацював і були виконані необхідні операції.




In [45]:
from sqlalchemy import text

def create_order_simple(customer_number: int,
                        items: list[dict],
                        status: str = "In Process",
                        comment: str | None = "Created from HW"):
    """
    items = [
      {"productCode": "S10_1678", "quantity": 3},   
      {"productCode": "S10_2016", "quantity": 2}
    ]
    """

    assert items and all(int(it["quantity"]) > 0 for it in items), "Список товарів має бути, qty > 0"

    with engine.begin() as cn:
        ok = cn.execute(text(
            "SELECT 1 FROM customers WHERE customerNumber=:cid"
        ), {"cid": customer_number}).scalar()
        if not ok:
            raise ValueError(f"Клієнта {customer_number} не знайдено")

        order_no = cn.execute(text(
            "SELECT COALESCE(MAX(orderNumber), 0) + 1 FROM orders"
        )).scalar()

        cn.execute(text("""
            INSERT INTO orders (orderNumber, orderDate, requiredDate, status, comments, customerNumber)
            VALUES (:no, CURDATE(), DATE_ADD(CURDATE(), INTERVAL 7 DAY), :st, :cm, :cid)
        """), {"no": order_no, "st": status, "cm": comment, "cid": customer_number})

    
        for i, it in enumerate(items, start=1):
            pc  = it["productCode"]
            qty = int(it["quantity"])

            row = cn.execute(text("""
                SELECT quantityInStock, MSRP
                FROM products
                WHERE productCode=:pc
            """), {"pc": pc}).mappings().first()

            if not row:
                raise ValueError(f"Товар {pc} не знайдено")
            if int(row["quantityInStock"]) < qty:
                raise ValueError(f"Недостатньо на складі для {pc}: є {row['quantityInStock']}, треба {qty}")

            price_each = float(it.get("priceEach") or row["MSRP"])

            cn.execute(text("""
                INSERT INTO orderdetails
                  (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
                VALUES (:no, :pc, :qty, :price, :line)
            """), {"no": order_no, "pc": pc, "qty": qty, "price": price_each, "line": i})

            cn.execute(text("""
                UPDATE products
                SET quantityInStock = quantityInStock - :qty
                WHERE productCode = :pc
            """), {"qty": qty, "pc": pc})

        return order_no

In [46]:
items = [
    {"productCode": "S10_1678", "quantity": 3},
    {"productCode": "S10_2016", "quantity": 2},
]

order_no = create_order_simple(customer_number=103, items=items,
                               status="In Process",
                               comment="Test order from homework")
order_no

10428

In [47]:
pd.read_sql(text("""
    SELECT orderNumber, orderDate, requiredDate, status, customerNumber, comments
    FROM orders WHERE orderNumber=:no
"""), engine, params={"no": order_no})

pd.read_sql(text("""
    SELECT orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber
    FROM orderdetails
    WHERE orderNumber=:no
    ORDER BY orderLineNumber
"""), engine, params={"no": order_no})

for it in items:
    pc = it["productCode"]
    print(f"Stock for {pc}:")
    display(pd.read_sql(text("""
        SELECT productCode, quantityInStock
        FROM products WHERE productCode=:pc
    """), engine, params={"pc": pc}))

Stock for S10_1678:


,productCode,quantityInStock
0,S10_1678,7924


Stock for S10_2016:


,productCode,quantityInStock
0,S10_2016,6619
